# Simple place cell statistics for search and homing path maps

We calculate the statistics of firirng rate maps of the search and homing paths.

We also get the map stability between conditions.

In [1]:
%load_ext autoreload
%autoreload 2
%run setup_project.py
prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList,pose_file_extension = ".pose_kf.npy")

Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|███████████████████████████████████████████| 39/39 [00:37<00:00,  1.05it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


Get intervals for the search and homing paths.

In [4]:
for ses, sSes in tqdm(zip(myProject.sessionList,sSesList)):
    getSearchHomingIntervals(ses,sSes)

39it [00:00, 51.13it/s]


In [5]:
ses.intervalDict.keys()

dict_keys(['circ80', 'circ80_1', 'circ80_2', 'task', 'task_1', 'task_2', 'light', 'light_1', 'light_2', 'dark', 'dark_1', 'dark_2', 'searchPath_light', 'searchPath_light_1', 'searchPath_light_2', 'searchPath_dark', 'searchPath_dark_1', 'searchPath_dark_2', 'searchToLeverPath_light', 'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2', 'homingPath_light', 'homingPath_light_1', 'homingPath_light_2', 'homingPath_dark', 'homingPath_dark_1', 'homingPath_dark_2', 'homingFromLeavingLever_light', 'homingFromLeavingLever_light_1', 'homingFromLeavingLever_light_2', 'homingFromLeavingLever_dark', 'homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2', 'atLever_light', 'atLever_light_1', 'atLever_light_2', 'atLever_dark', 'atLever_dark_1', 'atLever_dark_2', 'all_light', 'all_light_1', 'all_light_2', 'all_dark', 'all_dark_1', 'all_dark_2', 'all_light_noPress', 'all_dark_noPress', 'all_task', 'all_task_1

In [7]:
def getMapStats(ses,sSes,onlyArena=False):
    """
    Get all the map stats of the data within the intervals of interest
    """
    
    xy_range=np.array([[-50,-90],[50,60]])
    
    # get all the conditions we are interested
    
    conditionDicts = [{"type" : "searchToLeverPath", "light" : "light"},
                      {"type" : "searchToLeverPath", "light" : "light_1"},
                      {"type" : "searchToLeverPath", "light" : "light_2"},
                      {"type" : "searchToLeverPath", "light" : "dark"},
                      {"type" : "searchToLeverPath", "light" : "dark_1"},
                      {"type" : "searchToLeverPath", "light" : "dark_2"},
                      {"type" : "homingFromLeavingLever", "light" : "light"},
                      {"type" : "homingFromLeavingLever", "light" : "light_1"},
                      {"type" : "homingFromLeavingLever", "light" : "light_2"},
                      {"type" : "homingFromLeavingLever", "light" : "dark"},
                      {"type" : "homingFromLeavingLever", "light" : "dark_1"},
                      {"type" : "homingFromLeavingLever", "light" : "dark_2"}]
    
    res = []
    for d in conditionDicts:
    
        navPathType= d["type"]
        light = d["light"]
    
        myDict = {}
        # we can now get intervals for any navPath types!
        inter = ses.intervalDict[navPathType+"_"+light]

        sSes.ap.set_intervals(inter)
        
        if onlyArena:
            # this should come after setting the time intervals, will be reset when calling sSes.ap.set_intervals again
            sSes.ap.invalid_outside_spatial_area(shape="circle",radius=43.0,center=np.array([0,0])) ## remove the bridge and homebase

        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing_sigma_cm = 3, smoothing=True,xy_range=xy_range)
    
        # get peak rate from smoothed maps
        myDict["peakRate"] = [ np.nanmax(n.spatial_properties.firing_rate_map) for n in sSes.cg.neuron_list] # not affected by onlyArena!!!!
        myDict["meanRate"] = [ n.spike_train.mean_firing_rate() for n in sSes.cg.neuron_list]
        myDict["validBins"] = [ np.sum(~np.isnan(n.spatial_properties.firing_rate_map)) for n in sSes.cg.neuron_list] # not affected by onlyArena!!!!
        
        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing=False,xy_range=xy_range)
        
        myDict["info"] = [ n.spatial_properties.information_score() for n in sSes.cg.neuron_list]
        myDict["sparsity"] = [ n.spatial_properties.sparsity_score() for n in sSes.cg.neuron_list]
        myDict["condition"] =  navPathType+"_"+light
        myDict["session"] = sSes.name
        myDict["mouse"] = sSes.subject
        myDict["cellId"] = [sSes.name+"_"+n.name for n in sSes.cg.neuron_list]
        

        res.append(pd.DataFrame(myDict))
        
    # reset to original intervals
    sSes.ap.unset_intervals()
    
    return pd.concat(res)

## Try out on one session

In [8]:
ses = myProject.sessionList[0]
sSes = sSesList[0]

In [9]:
res = getMapStats(ses,sSes)

/home/kevin/repo/spikeA/spikeA/Spatial_properties.py:524: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))


In [10]:
res.head()

,peakRate,meanRate,validBins,info,sparsity,condition,session,mouse,cellId
0,47.284492,10.015704,194,0.402537,0.381859,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_80
1,30.322563,16.201361,194,0.447611,0.336624,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_90
2,53.661105,14.814169,194,0.304746,0.326180,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_92
3,13.225959,0.706683,194,4.014413,0.955115,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_96
4,13.343247,2.713314,194,0.553388,0.444280,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_98


## Run on all sessions

In [11]:
res = [ getMapStats(ses,sSes) for ses, sSes in tqdm(zip(myProject.sessionList,sSesList))]

0it [00:00, ?it/s]/home/kevin/repo/spikeA/spikeA/Spatial_properties.py:524: RuntimeWarning: invalid value encountered in double_scalars
  return 1-(((np.nansum(p*v))**2)/np.nansum(p*(v**2)))
39it [12:13, 18.82s/it]


In [12]:
df = pd.concat(res)
df

,peakRate,meanRate,validBins,info,sparsity,condition,session,mouse,cellId
0,47.284492,10.015704,194,0.402537,0.381859,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_80
1,30.322563,16.201361,194,0.447611,0.336624,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_90
2,53.661105,14.814169,194,0.304746,0.326180,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_92
3,13.225959,0.706683,194,4.014413,0.955115,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_96
4,13.343247,2.713314,194,0.553388,0.444280,searchToLeverPath_light,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_98
...,...,...,...,...,...,...,...,...,...
57,22.328482,1.542934,409,1.869248,0.802445,homingFromLeavingLever_dark_2,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_238
58,45.848396,16.513864,409,0.671661,0.496416,homingFromLeavingLever_dark_2,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_240
59,29.586391,3.320662,409,1.848411,0.834383,homingFromLeavingLever_dark_2,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_241
60,25.255490,3.152952,409,1.752594,0.801313,homingFromLeavingLever_dark_2,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_242


In [13]:
fn=fn=myProject.dataPath+"/results/mySearchHomingMapStats.csv"
print(fn)
df.to_csv(fn,index=False)

/adata/projects/autopi_ca1/results/mySearchHomingMapStats.csv


## Map similarity

We compare the map similarity across conditions.

In [14]:
from scipy.stats import pearsonr
def map_cor(a,b):
    """
    Correlation coefficient between two firing rate maps
    
    Arguments:
    a: 2D np.array (map1)
    b: 2D np.array (map2)
    
    Returns:
    Pearson correlation coefficient between a and b
    """
    a = a.flatten()
    b = b.flatten()
    indices = np.logical_and(~np.isnan(a), ~np.isnan(b))
    if np.sum(indices)<2:
        return np.nan
    r,p = pearsonr(a[indices],b[indices])
    return r

In [15]:
fn = "/adata/projects/autopi_ca1/results/mySearchHomingMaps.pickle" #, dictionary with key per condition
with open(fn, 'rb') as handle:
    myMaps = pickle.load(handle)

In [16]:
myMaps.keys()

dict_keys(['searchToLeverPath_light', 'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2', 'homingFromLeavingLever_light', 'homingFromLeavingLever_light_1', 'homingFromLeavingLever_light_2', 'homingFromLeavingLever_dark', 'homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2'])

In [17]:
env1 = ['searchToLeverPath_light','searchToLeverPath_dark','searchToLeverPath_light','homingFromLeavingLever_light', "searchToLeverPath_light","searchToLeverPath_dark", "searchToLeverPath_light_1","searchToLeverPath_dark_1","homingFromLeavingLever_light_1","homingFromLeavingLever_dark_1"]
env2 = ['homingFromLeavingLever_light','homingFromLeavingLever_dark','searchToLeverPath_dark', 'homingFromLeavingLever_dark', "homingFromLeavingLever_dark", "homingFromLeavingLever_light", "searchToLeverPath_light_2","searchToLeverPath_dark_2","homingFromLeavingLever_light_2","homingFromLeavingLever_dark_2"]
condNames = ["slhl","sdhd","slsd","hlhd","slhd","sdhl","sl1sl2","sd1sd2","hl1hl2","hd1hd2"]
list(zip(env1,env2,condNames))

[('searchToLeverPath_light', 'homingFromLeavingLever_light', 'slhl'),
 ('searchToLeverPath_dark', 'homingFromLeavingLever_dark', 'sdhd'),
 ('searchToLeverPath_light', 'searchToLeverPath_dark', 'slsd'),
 ('homingFromLeavingLever_light', 'homingFromLeavingLever_dark', 'hlhd'),
 ('searchToLeverPath_light', 'homingFromLeavingLever_dark', 'slhd'),
 ('searchToLeverPath_dark', 'homingFromLeavingLever_light', 'sdhl'),
 ('searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'sl1sl2'),
 ('searchToLeverPath_dark_1', 'searchToLeverPath_dark_2', 'sd1sd2'),
 ('homingFromLeavingLever_light_1',
  'homingFromLeavingLever_light_2',
  'hl1hl2'),
 ('homingFromLeavingLever_dark_1', 'homingFromLeavingLever_dark_2', 'hd1hd2')]

In [18]:
mapCorDf = pd.DataFrame()
nCells=myMaps[list(myMaps.keys())[0]].shape[0]
for e1,e2,c in zip(env1,env2,condNames):
    print(e1,e2,c)
    correlation  = [ map_cor(myMaps[e1][i],myMaps[e2][i]) for i in range(nCells) ]
    df1 = pd.DataFrame({"condition":c,
                       "r":correlation})
    mapCorDf = pd.concat([mapCorDf,df1])

searchToLeverPath_light homingFromLeavingLever_light slhl
searchToLeverPath_dark homingFromLeavingLever_dark sdhd
searchToLeverPath_light searchToLeverPath_dark slsd


/home/kevin/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


homingFromLeavingLever_light homingFromLeavingLever_dark hlhd
searchToLeverPath_light homingFromLeavingLever_dark slhd
searchToLeverPath_dark homingFromLeavingLever_light sdhl
searchToLeverPath_light_1 searchToLeverPath_light_2 sl1sl2
searchToLeverPath_dark_1 searchToLeverPath_dark_2 sd1sd2
homingFromLeavingLever_light_1 homingFromLeavingLever_light_2 hl1hl2
homingFromLeavingLever_dark_1 homingFromLeavingLever_dark_2 hd1hd2


In [19]:
mapCorDf

,condition,r
0,slhl,-0.310983
1,slhl,-0.232300
2,slhl,0.016335
3,slhl,0.165152
4,slhl,-0.575119
...,...,...
1263,hd1hd2,0.095677
1264,hd1hd2,0.563495
1265,hd1hd2,0.091580
1266,hd1hd2,-0.164801


In [20]:
fn=myProject.dataPath+"/results/searchHomingMapCorDf.csv"
print("Saving to",fn)
mapCorDf.to_csv(fn,index=False)

Saving to /adata/projects/autopi_ca1/results/searchHomingMapCorDf.csv
